# Atividades Práticas Supervisionadas(APS)

### Importando Bibiotecas

In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from nltk import word_tokenize
import nltk
import re
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer

C:\Users\rafael.nsouza\AppData\Local\anaconda3\envs\aps\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pré-processamento dos Dados

### Funções para limpar os textos

In [4]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

def Limpeza_dados(instancia):
    # Remover links
    instancia = re.sub(r'http\S+', '', instancia)
    # Remover menções a usuários
    instancia = re.sub(r'@\w+', '', instancia)
    # Remover caracteres especiais e números
    instancia = re.sub(r'[^a-zA-Z\s]', '', instancia)
    # Remover espaços extras
    instancia = re.sub(r'\s+', ' ', instancia)
    return instancia.strip()

wordnet_lemmatizer = WordNetLemmatizer()

def Lemmatization(instancia):
  palavras = []
  for w in instancia.split():
    palavras.append(wordnet_lemmatizer.lemmatize(w))
  return (" ".join(palavras))

def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [stemmer.stem(i) for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

### Carregar dados do arquivo e limpar os Textos

In [5]:
df = pd.read_csv("../../data/Tweets_Mg.csv")
df = df[["Text", "Classificacao"]]

df = df[df['Classificacao'] != 'Neutro']
df["Classificacao"] = df["Classificacao"].replace({'Negativo': 0, 'Positivo': 1})
df.drop_duplicates(['Text'], inplace=True)
tweets = df['Text'].apply(Limpeza_dados)
tweets = [Preprocessing(i) for i in tweets]
classes = df['Classificacao']

print("Número de tweets após a remoção:", len(tweets))
print("Número de rótulos após a remoção:", len(classes))



C:\Users\rafael.nsouza\AppData\Local\Temp\ipykernel_7844\4034209713.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Classificacao"] = df["Classificacao"].replace({'Negativo': 0, 'Positivo': 1})


Número de tweets após a remoção: 3798
Número de rótulos após a remoção: 3798


## Treinando o Modelo

### Carregar o tokenizer e o modelo pré-treinado BERT

In [6]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Conjunto de dados de Treinamento Adicional

In [7]:
train_sentences = tweets
train_labels = torch.tensor(classes.tolist())  # 1 para positivo, 0 para negativo

### Tokenizar os dados de treinamento

In [8]:
encoded_dict = tokenizer(train_sentences, padding=True, truncation=True, return_tensors='pt')
input_ids = encoded_dict['input_ids']
attention_mask = encoded_dict['attention_mask']

### Criar o conjunto de dados de treinamento

In [9]:
print(len(input_ids))
print(len(attention_mask))
print(len(train_labels))

3798
3798
3798


In [10]:
train_dataset = TensorDataset(input_ids, attention_mask, train_labels)

### Definir hiperparâmetros de treinamento

In [11]:
batch_size = 128
learning_rate = 2e-5
epochs = 20

### Criar o DataLoader para o conjunto de dados de treinamento

In [12]:
train_loader = DataLoader(train_dataset, batch_size=batch_size)

### Configurar otimizador e função de perda

In [13]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

C:\Users\rafael.nsouza\AppData\Local\anaconda3\envs\aps\lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Treinar o modelo

In [ ]:
print('0')
for epoch in range(epochs):
    print('1')
    model.train()
    total_loss = 0
    for batch in train_loader:
        print('2')
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}")


0
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Epoch 1/20, Loss: 19.03742466121912
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Epoch 2/20, Loss: 21.95818355679512
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Epoch 3/20, Loss: 20.39417166262865
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Epoch 4/20, Loss: 17.53788362443447
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Epoch 5/20, Loss: 21.228616192936897
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Epoch 6/20, Loss: 17.90257538855076
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Epoch 7/20, Loss: 20.75567853450775
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Epoch 8/20, Loss: 16.556241810321808
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
Epoch 9/20, Loss: 20.727863878011703
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


### Salvar o modelo treinado

In [ ]:
output_model_file = "sentiment_model.bin"
torch.save(model.state_dict(), output_model_file)

## Carregar o modelo pré-treinado BERT para análise de sentimentos

### Carregar os pesos do modelo treinado

In [ ]:
model.load_state_dict(torch.load("sentiment_model.bin"))
model.eval()

### Frases de teste

In [ ]:
test_sentences = [
    "A trama deste filme é envolvente.",
    "O desfecho deste livro me surpreendeu.",
    "As sobremesas deste restaurante são divinas.",
    "Este filme é um clássico!",
    "O enredo deste livro é cativante.",
    "A comida neste café é de dar água na boca.",
    "Este restaurante sempre me impressiona com seus pratos.",
    "O roteiro deste filme é impecável.",
    "Este livro me emocionou do começo ao fim.",
    "Os pratos deste restaurante são uma explosão de sabores.",
    "A atuação neste filme é excepcional.",
    "Este livro é uma obra-prima.",
    "A experiência neste restaurante é única.",
    "Os personagens deste filme são muito bem desenvolvidos.",
    "O autor deste livro tem uma escrita cativante.",
    "A atmosfera deste restaurante é acolhedora.",
    "Este filme me fez refletir sobre muitas coisas.",
    "Este livro é uma verdadeira viagem.",
    "As críticas deste filme são merecidas.",
    "O final deste livro me deixou sem palavras.",
    "Este restaurante é uma excelente escolha para ocasiões especiais.",
    "Os efeitos especiais neste filme são impressionantes.",
    "Este livro é uma leitura obrigatória.",
    "A decoração deste restaurante é encantadora.",
    "A trilha sonora deste filme é incrível.",
    "Este livro me prendeu do início ao fim.",
    "A apresentação dos pratos neste restaurante é impecável.",
    "A mensagem deste filme é poderosa.",
    "Este livro me fez repensar muitas coisas.",
    "A comida neste restaurante é de outro mundo."
]

### Tokenizar as frases de teste

In [ ]:
encoded_dict = tokenizer(test_sentences, padding=True, truncation=True, return_tensors='pt')
input_ids = encoded_dict['input_ids']
attention_mask = encoded_dict['attention_mask']

### Realizar as previsões

In [ ]:
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

### Mapear as previsões de volta para rótulos de sentimentos

In [ ]:
predictions = torch.argmax(logits, dim=1)
sentiment_labels = ['Positivo' if pred == 1 else 'Negativo' for pred in predictions]

### Exibir os resultados

In [ ]:
for sentence, sentiment in zip(test_sentences, sentiment_labels):
    print(f"Frase: {sentence}")
    print(f"Sentimento: {sentiment}\n")